In [18]:
# Import and associate Google Drive with Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import librosa
import numpy as np

In [20]:
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('/content/drive/MyDrive/Colab Notebooks/Audio Nagham Recognition/trained_models/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/Colab Notebooks/Audio Nagham Recognition/trained_models/model_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [22]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/Audio Nagham Recognition/trained_models/scaler2.pickle', 'rb') as f:
    scaler2 = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/Audio Nagham Recognition/trained_models/encoder2.pickle', 'rb') as f:
    encoder2 = pickle.load(f)

print("Done")

Done


In [23]:
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])

    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [24]:
def get_predict_feat(path):
    d, s_rate= librosa.load(path, duration=35.0, offset=0.3)
    res=extract_features(d)
    result=np.array(res)
    target_shape = (1,33176)
    zeroes_array = np.zeros(target_shape)
    zeroes_array[0, :result.size] = result
    i_result = scaler2.transform(zeroes_array)
    final_result=np.expand_dims(i_result, axis=2)

    return final_result

In [25]:
category_naghams={

          # Bayati ashli
          1:'Bayati Ashli Qorror',
          # Bayati nawa
          2:'Bayati Ashli Nawa',
          # Bayati syuti
          3:'Bayati Syuri',
          4:'Bayati Syuri',
          5:'Bayati Syuri',
          # Bayati husaini
          6:'Bayati Husaini',
          7:'Bayati Husaini',
          8:'Bayati Husaini',
          9:'Bayati Husaini',
          10:'Bayati Husaini',
          11:'Bayati Husaini',
          # Bayati ashli jawab
          12:'Bayati Ashli Jawab',
          13:'Bayati Ashli Jawab',
          14:'Bayati Ashli Jawab',
          # Bayati syuri jawabuljawab
          15:'Bayati Syuri Jawabuljawab',
          47:'Bayati Syuri Jawabuljawab',
          # Shaba
          16:'Shaba Ashli',
          17:'Shaba Ashli',
          18:'Shaba Ashli',
          19:'Shaba Ashli',
          # Jawab shaba
          20:'Jawab Shaba',
          # Jawab shaba maalajam
          21:'Jawab Shaba Maalajam',
          # Jawab shaba maalbastanjar
          22:'Jawab Shaba Maalbastanjar',
          # Hijaz ashli
          23:'Hijaz Ashli',
          24:'Hijaz Ashli',
          25:'Hijaz Ashli',
          26:'Hijaz Ashli',
          27:'Hijaz Ashli',
          28:'Hijaz Ashli',
          # Hijaz kar
          29:'Hijaz Kar',
          # Hijaz karkur
          30:'Hijaz Karkur',
          # Hijaz kur
          31:'Hijaz Kur',
          # Rast ashli
          32:'Rast Ashli',
          33:'Rast Ashli',
          34:'Rast Ashli',
          35:'Rast Ashli',
          # Rast alanawa
          36:'Rast Alanawa',
          37:'Rast Alanawa',
          38:'Rast Alanawa',
          # Rast zunjiran
          48:'Rast zunjiran',
          # Sika ashli
          39:'Sika Ashli',
          # Sika turki
          40:'Sika Turki',
          # Sika mishri
          41:'Sika Mishri',
          # Jiharkah ashli
          42:'Jiharkah Ashli',
          # Jiharkah jawab
          43:'Jiharkah Jawab',
          44:'Jiharkah Jawab',
          # Nahawand ashli
          45:'Nahawand Ashli',
          # Nahawand jawab
          46:'Nahawand Jawab'

          }

In [26]:
pip install pydub

In [27]:
from pydub import AudioSegment

In [28]:
def fix_and_predict(file_path):
    fixed_file_path = fix_wav(file_path)
    res = get_predict_feat(fixed_file_path)
    predictions = loaded_model.predict(res)
    nagham_index = np.argmax(predictions[0])
    nagham_label = category_naghams[nagham_index + 1]
    print("Predicted:", nagham_label)

In [29]:
def fix_wav(file_path):
    audio = AudioSegment.from_file(file_path)
    audio.export(file_path, format="wav")
    return file_path

In [30]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Audio Nagham Recognition/dataset/A.1_tausyih_bayati_ashli_qorror_ar-rahman/1.0_ar-rahman_bayyati_ashli_qorror_v1_b.wav'
fix_and_predict(file_path)

1/1 [==============================] - 0s 306ms/step
Predicted: Bayati Husaini
